# 0. Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader
import torch.nn as nn


from helpers import read_hdf5, CombineDataset
from lenshelpers import Residual

from model.helpers_model import NeuralNet

import warnings
warnings.filterwarnings('ignore') 

# 1. Building the data set

In [2]:
size = 60000
ratio = 0.75; percent = 0.01

for i in np.arange(1,4):
    res = Residual()
    res.build(size, i, ratio = ratio, per_error = percent)

print('Data Generation Finished')

metadata = pd.DataFrame()

for i in np.arange(1,4):
    ID_images = "E"+str(i)+"P"+str(int(percent*100))+"R"+str(int(ratio*100))
    [img, meta] = read_hdf5(ID_images, path = "data/dataSet/")
    metadata = pd.concat([metadata,meta], ignore_index=True)
    if i == 1:
        final_array = img
    else:
         final_array = np.concatenate((final_array, img))
metadata ['ID'] = np.arange(0,final_array.shape[0])
data_set = CombineDataset(metadata,'ID','class',final_array)

data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)

batch_size = 50; max_epoch = 100


loader_train = DataLoader(data_train, batch_size = batch_size, 
                          num_workers = 2, drop_last=True)
loader_test = DataLoader(data_test, batch_size = batch_size, 
                         num_workers = 2, drop_last=True)
print('Reading Data Finished')

Data Generation Finished


# 2. Neural network
## 1. Baseline
**Results** : _ epochs - _ accuracy

In [4]:
# epoch: 29.000, auc: 0.859
test_acc = np.zeros(max_epoch); 
netbasic = NeuralNet('BasicCNN', 'Adam')
for epoch in range(max_epoch):  # loop over the dataset multiple times
    netbasic.train(loader_train)
    res = netbasic.test(loader_test,epoch)

netbasic.load_checkpoint()
net_name = "basic_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)

print('Finished Training - Baseline')

## 2. AlexNet
**Results** : _ epochs - _ accuracy

In [5]:
test_acc = np.zeros(max_epoch); 
# epoch: 10 - accuracy: 0.838
netbasic = NeuralNet('AlexNet', 'Adam')
netbasic.load_checkpoint()
for epoch in range(23,max_epoch):
    netbasic.train(loader_train)
    res = netbasic.test(loader_test,epoch)
    print(str(epoch)+"\n")

netbasic.load_checkpoint()
net_name = "alexnet_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)
   
print('Finished Training - AlexNet')

epoch: 0.000, auc: 0.564
epoch: 26.000, auc: 0.570


## 3. ResNet18
**Results** : _ epochs - _ accuracy

In [ ]:
test_acc = np.zeros(max_epoch);
#epoch: 3.000, auc: 0.856
netbasic = NeuralNet('ResNet18', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train, resize_tp = 'Padding')
    res = netbasic.test(loader_test,epoch)
    print(str(epoch))
netbasic.load_checkpoint()
net_name = "resnet18_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)
   
print('Finished Training - ResNet18')

## 4. VGG16
**Results** : _ epochs - _ accuracy

In [ ]:
test_acc = np.zeros(max_epoch); 
netbasic = NeuralNet('VGG16', 'Adam')
#epoch: 2.000, auc: 0.845
for epoch in range(6,max_epoch):
    netbasic.train(loader_train, resize_tp = 'Padding')
    res = netbasic.test(loader_test,epoch)
    print(str(epoch))

netbasic.load_checkpoint()
net_name = "VGG16_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)
   
print('Finished Training - VGG16')

## 5. DenseNet161
**Results** : _ epochs - _ accuracy

In [ ]:
test_acc = np.zeros(max_epoch)

netbasic = NeuralNet('DenseNet161', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train, resize_tp = 'Padding')
    res = netbasic.test(loader_test,epoch)
    print(str(epoch))

netbasic.load_checkpoint()
net_name = "dense121_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)
   
print('Finished Training - DenseNet161')

## 6. GoogleNet
**Results** : _ epochs - _ accuracy

In [ ]:
test_acc = np.zeros(max_epoch)

netbasic = NeuralNet('GoogleNet', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train, resize_tp = 'Padding')
    res = netbasic.test(loader_test,epoch)
    print(str(epoch)+"\n")

netbasic.load_checkpoint() 
net_name = "googleNet_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)
   
print('Finished Training - GoogleNet')

## 7. SqueezeNet
**Results** : _ epochs - _ accuracy

In [ ]:
test_acc = np.zeros(max_epoch)

netbasic = NeuralNet('SqueezeNet', 'Adam')
for epoch in range(max_epoch):
    netbasic.train(loader_train, resize_tp = 'Padding')
    res = netbasic.test(loader_test,epoch)
    print(str(epoch)+"\n")

netbasic.load_checkpoint()
net_name = "squeezeNet_res.pt"
ID_net = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
path = "data/trainedNN/" 
torch.save(netbasic.net.state_dict(), path+ID_net+net_name)
   
print('Finished Training')